In [ ]:

# Install a networkx package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install networkx --upgrade --quiet

In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%run ../../load_magic/lists.py
from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 SequenceMatcher	 attempt_to_pickle	 bs	 
check_4_doubles	 check_for_typos	 csv	 encoding	 example_iterrows	 get_column_descriptions	 get_page_tables	 load_csv	 load_dataframes	 
load_object	 math	 nx	 os	 pd	 pickle	 re	 save_dataframes	 similar	 
store_objects	 sys	 time	 urllib	 


In [2]:

def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = bs(page_html, 'html.parser')
    
    return page_soup

In [3]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_counties_by_U.S._state_and_territory'
page_soup = get_page_soup(tables_url)

In [4]:

content_xpath = '/html/body/div[1]/div[1]/div[1]/div[3]/div[4]/div'
content_css = '#mw-content-text > div'
content_soup = page_soup.select(content_css)[0]

In [5]:

h2_list = []
for h2 in content_soup.select('h2'):
    print([fn for fn in dir(h2) if 'sib' in fn.lower()])
    break

['fetchNextSiblings', 'fetchPreviousSiblings', 'findNextSibling', 'findNextSiblings', 'findPreviousSibling', 'findPreviousSiblings', 'find_next_sibling', 'find_next_siblings', 'find_previous_sibling', 'find_previous_siblings', 'nextSibling', 'nextSiblingGenerator', 'next_sibling', 'next_siblings', 'previousSibling', 'previousSiblingGenerator', 'previous_sibling', 'previous_siblings']


In [6]:

print([fn for fn in dir(h2) if 'child' in fn.lower()])

['_lastRecursiveChild', 'childGenerator', 'children', 'findChild', 'findChildren', 'recursiveChildGenerator', 'replaceWithChildren', 'replace_with_children']


In [7]:

counties_regex = re.compile(r'The\s+([^0-9]+)\s+comprises\s+(\d+)\s+(counties|organized boroughs|parishes)')
rows_list = []
columns_list = ['state_str', 'counties_count', 'county_str']
for child in content_soup.children:
    if child.name == 'h2':
        comprises_str = list(child.next_siblings)[1].text.strip()
        match_obj = counties_regex.search(comprises_str)
        if match_obj:
            state_str = match_obj.group(1)
            counties_count = int(match_obj.group(2))
            county_str = match_obj.group(3)
            #print('"{}": {}'.format(comprises_str, counties_count))
            row_dict = {}
            row_dict['state_str'] = state_str
            row_dict['counties_count'] = counties_count
            row_dict['county_str'] = county_str
            rows_list.append(row_dict)
        #print([fn for fn in dir(comprises_p)])
        #break
counties_df = pd.DataFrame(rows_list, columns=columns_list)
print(counties_df['county_str'].unique().tolist())
counties_df.sort_values('counties_count').head(10)

['counties', 'organized boroughs', 'parishes']


,state_str,counties_count,county_str
7,State of Delaware,3,counties
10,State of Hawaii,5,counties
38,State of Rhode Island and Providence Plantations,5,counties
6,State of Connecticut,8,counties
28,State of New Hampshire,10,counties
20,Commonwealth of Massachusetts,14,counties
44,State of Vermont,14,counties
2,State of Arizona,15,counties
18,State of Maine,16,counties
27,State of Nevada,16,counties


In [8]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'
df_list = get_page_tables(tables_url)

[(0, (60, 12)), (3, (11, 2)), (1, (9, 12)), (2, (4, 12))]


In [9]:

states_area_df = df_list[0].copy()
#print([fn for fn in dir(states_area_df.columns) if 'to_' in fn.lower()])
states_area_df.columns = ['state_federal_district_or_territory', 'total_area_rank', 'total_area_sq_mi', 'total_area_km_sq',
                          'land_area_rank', 'land_area_sq_mi',
                          'land_area_km_sq', 'land_area_percent_land', 'water_rank', 'water_sq_mi', 'water_km_sq', 'water_percent_water']
states_area_df['state_federal_district_or_territory'] = states_area_df['state_federal_district_or_territory'].map(lambda x: re.sub(r'\[[^\]\[]+\]',
                                                                                                                                   '', str(x)))
states_area_df = states_area_df.set_index('state_federal_district_or_territory')
states_area_df.sample(10).T

state_federal_district_or_territory,Indiana,Maryland,Virginia,Michigan,Illinois,50 states and District of Columbia,Ohio,Contiguous United States,Iowa,Nevada
total_area_rank,38,42,35,11,25,Total,34,Total,26,7
total_area_sq_mi,36419.6,12405.9,42774.9,96713.5,57913.6,3.79674e+06,44825.6,3.12043e+06,56272.8,110572
total_area_km_sq,94326,32131,110787,250487,149995,9833517,116098,8081867,145746,286380
land_area_rank,38,42,36,22,24,NaN,35,NaN,23,7
land_area_sq_mi,35826.1,9707.24,39490.1,56538.9,55518.9,3.53191e+06,40860.7,2.95484e+06,55857.1,109781
land_area_km_sq,92789,25142,102279,146435,143793,9147593,105829,7653004,144669,284332
land_area_percent_land,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
water_rank,39,18,15,2,19,NaN,14,NaN,45,36
water_sq_mi,593.44,2698.69,3284.84,40174.61,2394.62,264836.79,3964.89,165584.6,415.68,790.65
water_km_sq,1537,6990,8508,104052,6202,685924,10269,428862,1077,2048


In [10]:

left_list = counties_df['state_str'].tolist()
right_list = states_area_df.index.tolist()
typos_df = check_for_typos(left_list, right_list)
match_series = (typos_df['max_similarity'] >= 0.6)
counties_df['state_federal_district_or_territory'] = pd.np.nan
for row_index, row_series in typos_df[match_series].iterrows():
    state_str = row_series['left_item']
    state_federal_district_or_territory = row_series['right_item']
    match_series = (counties_df['state_str'] == state_str)
    counties_df.loc[match_series, 'state_federal_district_or_territory'] = state_federal_district_or_territory

In [11]:

counties_df.sort_values('state_federal_district_or_territory')

,state_str,counties_count,county_str,state_federal_district_or_territory
0,State of Alabama,67,counties,Alabama
2,State of Arizona,15,counties,Arizona
3,State of Arkansas,75,counties,Arkansas
4,State of California,58,counties,California
5,State of Colorado,64,counties,Colorado
6,State of Connecticut,8,counties,Connecticut
7,State of Delaware,3,counties,Delaware
8,State of Florida,67,counties,Florida
9,State of Georgia,159,counties,Georgia
12,State of Illinois,102,counties,Illinois


In [12]:

counties_regex = re.compile('(State|Commonwealth) of ([A-Za-z ]+)')
right_list = states_area_df.index.tolist()
counties_df['state_type'] = pd.np.nan
counties_df['formal_name'] = pd.np.nan
def g(row_series):
    state_str = row_series['state_str']
    match_obj = counties_regex.search(state_str)
    if match_obj:
        row_series['state_type'] = match_obj.group(1)
        row_series['formal_name'] = match_obj.group(2)
        typos_df = check_for_typos([row_series['formal_name']], right_list)
        state_name = typos_df.sort_values('max_similarity', ascending=False).head(1)['right_item'].tolist()[0]
        row_series['state_federal_district_or_territory'] = state_name
    
    return row_series

In [13]:

counties_df = counties_df.apply(g, axis=1)
counties_df = counties_df.set_index('state_federal_district_or_territory')
counties_df.sample(10).T

state_federal_district_or_territory,New Mexico,New Jersey,Texas,Arkansas,Louisiana,Washington,Idaho,Michigan,Nebraska,Mississippi
state_str,State of New Mexico,State of New Jersey,State of Texas,State of Arkansas,State of Louisiana,State of Washington,State of Idaho,State of Michigan,State of Nebraska,State of Mississippi
counties_count,33,21,254,75,64,39,44,83,93,82
county_str,counties,counties,counties,counties,parishes,counties,counties,counties,counties,counties
state_type,State,State,State,State,State,State,State,State,State,State
formal_name,New Mexico,New Jersey,Texas,Arkansas,Louisiana,Washington,Idaho,Michigan,Nebraska,Mississippi


In [51]:

us_states_df = pd.merge(left=states_area_df, right=counties_df, left_index=True, right_index=True, suffixes=('_area', '_counties'))
columns_list = ['total_area_rank', 'total_area_sq_mi', 'total_area_km_sq', 'land_area_rank', 'land_area_sq_mi',
                'land_area_km_sq', 'water_rank', 'water_sq_mi', 'water_km_sq',
                'state_str', 'counties_count', 'county_str', 'state_type']
us_states_df = us_states_df[columns_list]
columns_list = ['total_area_rank', 'total_area_sq_mi', 'total_area_km_sq', 'land_area_rank', 'land_area_sq_mi',
                'land_area_km_sq', 'water_rank', 'water_sq_mi', 'water_km_sq',
                'formal_name', 'counties_count', 'county_type', 'state_type']
us_states_df.columns = columns_list

In [52]:

def f(x):
    if x == 'organized boroughs':
        county_type = 'Boroughs'
    elif x == 'parishes':
        county_type = 'Parishes'
    else:
        county_type = 'Counties'
    
    return county_type

us_states_df['county_type'] = us_states_df['county_type'].map(f)

In [53]:

state_merge_df = load_object('state_merge_df')
#print(state_merge_df.columns.tolist())
state_merge_df.columns = ['white_percent', 'black_percent', 'hispanic_percent', 'asian_percent', 'native_percent',
                          'islander_percent', 'multi_percent', 'racial_gini_index', 'effectiveness_rank', 'health_care_score',
                          'education_score', 'economy_score', 'infrastructure_score', 'opportunity_score',
                          'fiscal_stability_score', 'crime_corrections_score', 'natural_environment_score', 'state_abbreviation',
                          'gdp_rank', 'gdp_2018', 'gdp_percent', 'state_region', 'homicide_rate_2018', 'homicide_rate_2017',
                          'homicide_rate_2014', 'homicide_rate_2010', 'homicide_rate_2005', 'homicide_rate_2000', 'homicide_rate_1996']
us_states_df = pd.merge(left=state_merge_df, right=us_states_df, left_index=True, right_index=True, suffixes=('_merge', '_counties'))
store_objects(us_states_df=us_states_df)
us_states_df.sample(10).T

Pickling to C:\Users\dev\Documents\repositories\notebooks\StatsByCountry\saves\pickle\us_states_df.pickle


,South Dakota,Massachusetts,Connecticut,Virginia,West Virginia,Oregon,Colorado,North Dakota,New Jersey,Indiana
white_percent,0.83,0.72,0.67,0.62,0.93,0.76,0.68,0.85,0.55,0.79
black_percent,0.02,0.07,0.1,0.19,0.04,0.02,0.04,0.03,0.13,0.09
hispanic_percent,0.04,0.12,0.16,0.09,0.01,0.13,0.22,0.03,0.21,0.07
asian_percent,0.01,0.06,0.05,0.07,0.01,0.05,0.03,0.02,0.1,0.02
native_percent,0.08,0,0,0,0,0.01,0.01,0.06,0,0
islander_percent,0,0,0,0,0,0,0,0,0,0
multi_percent,0.02,0.03,0.03,0.03,0.02,0.04,0.03,0.02,0.02,0.02
racial_gini_index,0.757143,0.697143,0.678925,0.657143,0.81471,0.721358,0.698727,0.746818,0.61669,0.750361
effectiveness_rank,20,8,21,7,47,27,10,15,12,36
health_care_score,32,2,3,18,48,17,12,38,6,40


In [54]:

us_states_df = load_object('us_states_df')
column_descriptions_df = get_column_descriptions(us_states_df)
match_series = (column_descriptions_df['dtype'] == 'float64') & (column_descriptions_df['only_integers'] == True)
for column_name in column_descriptions_df[match_series]['column_name']:
    us_states_df[column_name] = us_states_df[column_name].map(lambda x: int(x))
us_states_df['total_area_rank'] = us_states_df['total_area_rank'].map(lambda x: int(x))
us_states_df['water_sq_mi'] = us_states_df['water_sq_mi'].map(lambda x: float(x))
us_states_df['water_km_sq'] = us_states_df['water_km_sq'].map(lambda x: int(x))
store_objects(us_states_df=us_states_df)

Pickling to C:\Users\dev\Documents\repositories\notebooks\StatsByCountry\saves\pickle\us_states_df.pickle


In [55]:

column_descriptions_df = get_column_descriptions(us_states_df)
match_series = (column_descriptions_df['dtype'] == 'object')
column_descriptions_df[match_series]

,column_name,dtype,count_blanks,count_uniques,count_zeroes,has_dates,min_value,max_value,only_integers
37,state_abbreviation,object,0,50,0,False,AK,WY,NaN
38,state_region,object,0,4,0,False,Midwest,West,NaN
39,formal_name,object,0,50,0,False,Commonwealth of Kentucky,State of Wyoming,NaN
40,county_type,object,0,3,0,False,Boroughs,Parishes,NaN
41,state_type,object,0,2,0,False,Commonwealth,State,NaN


In [77]:

columns_list = ['white_percent', 'black_percent', 'hispanic_percent', 'asian_percent', 'native_percent', 'islander_percent',
                'multi_percent', 'racial_gini_index', 'effectiveness_rank', 'health_care_score', 'education_score',
                'economy_score', 'infrastructure_score', 'opportunity_score', 'fiscal_stability_score',
                'crime_corrections_score', 'natural_environment_score', 'state_abbreviation', 'gdp_rank', 'gdp_2018', 'gdp_percent',
                'state_region', 'homicide_rate_2018', 'homicide_rate_2017', 'homicide_rate_2014', 'homicide_rate_2010',
                'homicide_rate_2005', 'homicide_rate_2000', 'homicide_rate_1996', 'total_area_rank',
                'total_area_km_sq', 'land_area_rank', 'land_area_km_sq', 'water_rank',
                'water_km_sq', 'formal_name', 'counties_count', 'county_type', 'state_type']
us_states_df = us_states_df[columns_list]
store_objects(us_states_df=us_states_df)

Pickling to C:\Users\dev\Documents\repositories\notebooks\StatsByCountry\saves\pickle\us_states_df.pickle


In [82]:

def g(row_series):
    total_area_km_sq = row_series['total_area_km_sq']
    counties_count = row_series['counties_count']
    
    return total_area_km_sq / counties_count

us_states_df['ave_county_area'] = us_states_df.apply(g, axis=1)
store_objects(us_states_df=us_states_df)
us_states_df.sort_values('ave_county_area', ascending=False).head(10).T

Pickling to C:\Users\dev\Documents\repositories\notebooks\StatsByCountry\saves\pickle\us_states_df.pickle


,Alaska,Arizona,Nevada,Wyoming,New Mexico,Utah,California,Oregon,Montana,Maine
white_percent,0.61,0.55,0.49,0.84,0.37,0.79,0.37,0.76,0.87,0.94
black_percent,0.03,0.04,0.09,0.01,0.02,0.01,0.05,0.02,0,0.01
hispanic_percent,0.07,0.31,0.29,0.1,0.49,0.14,0.39,0.13,0.04,0.02
asian_percent,0.07,0.03,0.08,0.01,0.01,0.02,0.14,0.05,0.01,0.01
native_percent,0.13,0.04,0.01,0.02,0.09,0.01,0,0.01,0.06,0.01
islander_percent,0.01,0,0.01,0,0,0.01,0,0,0,0
multi_percent,0.09,0.03,0.04,0.02,0.02,0.02,0.03,0.04,0.03,0.02
racial_gini_index,0.571429,0.634286,0.579915,0.774286,0.645714,0.745714,0.588921,0.721358,0.780764,0.806223
effectiveness_rank,44,34,37,31,46,4,19,27,29,32
health_care_score,25,23,31,42,34,9,7,17,28,19


In [89]:

import statsmodels.api as sm

def get_max_rsquared_adj(df, columns_list, verbose=False):
    if verbose:
        t0 = time.time()
    rows_list = []
    n = len(columns_list)
    for i in range(n):
        first_column = columns_list[i]
        first_series = df[first_column]
        max_similarity = 0.0
        max_column = first_column
        for j in range(n):
            second_column = columns_list[j]
            if first_column != second_column:
                second_series = df[second_column]

                # Assume the first column is never identical to the second column
                X, y = first_series.values.reshape(-1, 1), second_series.values.reshape(-1, 1)
                #this_similarity = abs(first_series.cov(second_series))

                # Compute with statsmodels, by adding intercept manually
                X1 = sm.add_constant(X)
                result = sm.OLS(y, X1).fit()
                this_similarity = abs(result.rsquared_adj)

                if this_similarity > max_similarity:
                    max_similarity = this_similarity
                    max_column = second_column

        # Get input row in dictionary format; key = col_name
        row_dict = {}
        row_dict['first_column'] = first_column
        row_dict['second_column'] = max_column
        row_dict['max_similarity'] = max_similarity

        rows_list.append(row_dict)

    column_list = ['first_column', 'second_column', 'max_similarity']
    column_similarities_df = pd.DataFrame(rows_list, columns=column_list)
    if verbose:
        t1 = time.time()
        print(t1-t0, time.ctime(t1))

    return column_similarities_df

In [91]:

us_states_df = load_object('us_states_df')
column_descriptions_df = get_column_descriptions(us_states_df)
match_series = (column_descriptions_df['dtype'] != 'object')
numeric_columns_list = column_descriptions_df[match_series].column_name.tolist()
covariance_df = get_max_rsquared_adj(df=us_states_df, columns_list=numeric_columns_list, verbose=False)
#match_series = (covariance_df['max_similarity'] >= 0.95)
match_series = (covariance_df['first_column'] == 'ave_county_area')
covariance_df[match_series].sort_values('max_similarity', ascending=False)

,first_column,second_column,max_similarity
34,ave_county_area,total_area_km_sq,0.802879


In [62]:

from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm_formula
import random

us_states_df = load_object('us_states_df')
column_descriptions_df = get_column_descriptions(us_states_df)
match_series = (column_descriptions_df['dtype'] != 'object')
numeric_columns_list = column_descriptions_df[match_series].column_name.tolist()
df = us_states_df[numeric_columns_list].copy()

model = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
x_column_name = random.choice(numeric_columns_list)
y_column_name = random.choice(numeric_columns_list)
X, y = df[x_column_name].values.reshape(-1, 1), df[y_column_name].values.reshape(-1, 1)
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [65]:

# Compute with formulas from the theory
yhat = model.predict(X)
SS_Residual = sum((y-yhat)**2)
SS_Total = sum((y-pd.np.mean(y))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
print(r_squared[0], adjusted_r_squared[0])

0.21029518012493809 0.193842996377541


In [66]:

# Compute with sklearn linear_model, although could not find any function to compute adjusted-r-square directly from documentation
print(model.score(X, y), 1 - (1-model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1))

0.21029518012493878 0.19384299637754154


In [67]:

# Compute with statsmodels, by adding intercept manually
X1 = sm.add_constant(X)
result = sm.OLS(y, X1).fit()
print(result.rsquared, result.rsquared_adj)

0.21029518012493875 0.19384299637754165


In [68]:

# Compute with statsmodels, another way, using formula
result = sm_formula.ols(formula='{} ~ {}'.format(y_column_name, x_column_name), data=df).fit()
#print(result.summary())
print(result.rsquared, result.rsquared_adj)

0.21029518012493875 0.19384299637754165
